In [ ]:
!pip install torch torchaudio librosa transformers pandas scikit-learn numpy openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.8 MB/

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dementia_path = "/content/drive/MyDrive/fyp/dementia"
non_dementia_path = "/content/drive/MyDrive/fyp/nodementia"

In [ ]:
print("Dementia Files:", os.listdir(dementia_path)[:5])
print("Non-Dementia Files:", os.listdir(non_dementia_path)[:5])

Dementia Files: ['Dan Ingram', 'Terry Jones', 'Maureen Forrester', 'Aileen Hernandez', 'Steve Lawrence']
Non-Dementia Files: ['Barry Levinson', 'Robert Wagner', 'Bill Wyman', 'Patrick Stewart', 'Paul Verhoeven']


In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
import whisper  # OpenAI Whisper ASR

# Load Whisper Model
whisper_model = whisper.load_model("medium")

def transcribe_audio(audio_path):
    result = whisper_model.transcribe(audio_path)
    return result["text"]

100%|██████████████████████████████████████| 1.42G/1.42G [00:14<00:00, 107MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [ ]:
# Function to extract speech features
def extract_speech_features(audio_path):
    y, sr = librosa.load(audio_path, sr=16000)
    return {
        "mfcc": np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1),
        "pitch": np.mean(librosa.yin(y, fmin=50, fmax=300)),  # Specify fmin and fmax as keyword arguments
        "pause_count": len(librosa.effects.split(y)),
        "speech_rate": len(librosa.onset.onset_detect(y=y, sr=sr))
    }

In [ ]:
def process_audio_data(directory, label):
    patient_features = []
    for patient_folder in os.listdir(directory):
        patient_path = os.path.join(directory, patient_folder)
        if os.path.isdir(patient_path):
            all_features = []
            transcriptions = []
            for file in os.listdir(patient_path):
                if file.endswith(".wav"):
                    audio_path = os.path.join(patient_path, file)
                    features = extract_speech_features(audio_path)
                    all_features.append(features)
                    transcriptions.append(transcribe_audio(audio_path))
            avg_features = {k: np.mean([f[k] for f in all_features]) for k in all_features[0]}
            avg_features["patient_id"] = patient_folder
            avg_features["transcription"] = " ".join(transcriptions)
            avg_features["label"] = label
            patient_features.append(avg_features)
    return pd.DataFrame(patient_features)

In [ ]:
dementia_df = process_audio_data("/content/drive/MyDrive/fyp/dementia", label=1)
healthy_df = process_audio_data("/content/drive/MyDrive/fyp/nodementia", label=0)

In [ ]:
import pandas as pd
speech_df = pd.concat([dementia_df, healthy_df], ignore_index=True)
speech_df.to_csv("speech_features.csv", index=False)

In [ ]:
speech_df.to_csv("/content/drive/MyDrive/fyp/speech_features.csv", index=False)

In [ ]:
speech_df = pd.read_csv("/content/drive/MyDrive/fyp/speech_features.csv")

# Verify data
print(speech_df.head())

        mfcc       pitch  pause_count  speech_rate         patient_id  \
0  -7.922390  168.733675          1.0   323.000000         Dan Ingram   
1 -20.059917  160.364489          1.0   339.000000        Terry Jones   
2 -14.480901  182.225977          1.0   403.000000  Maureen Forrester   
3 -19.103064  193.957124          2.0   394.666667   Aileen Hernandez   
4 -10.819764  125.532001          1.0   631.000000     Steve Lawrence   

                                       transcription  label  
0   Long before the blackout. All right. Yeah, bl...      1  
1   Well, yeah, but then it was the government th...      1  
2   You know, if you've ever gone to a place, a h...      1  
3   This is not going to sound like very ladylike...      1  
4   really really well at uh... you know i have u...      1  


In [ ]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
def get_text_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

In [ ]:
# Apply BERT to transcriptions
speech_df["bert_embedding"] = speech_df["transcription"].apply(lambda x: get_text_embedding(x).flatten())

In [ ]:
# Convert embeddings into separate columns
text_features = pd.DataFrame(speech_df["bert_embedding"].to_list(), columns=[f"text_dim_{i}" for i in range(768)])
speech_df = pd.concat([speech_df, text_features], axis=1).drop(columns=["bert_embedding", "transcription"])

In [ ]:
speech_df.to_csv("/content/drive/MyDrive/fyp/speech_with_bert.csv", index=False)

In [ ]:
import pandas as pd
speech_df = pd.read_csv("/content/drive/MyDrive/fyp/speech_with_bert.csv")

In [ ]:
import os
file_path = "/content/drive/MyDrive/fyp/speech_with_bert.csv"
if os.path.exists(file_path):
    speech_df = pd.read_csv(file_path)
    print("Loaded saved embeddings ✅")
else:
    print("Extracting BERT embeddings... 🕒")
    # Run BERT extraction here

Loaded saved embeddings ✅


In [ ]:
import numpy as np
np.save("/content/drive/MyDrive/fyp/bert_embeddings.npy", text_features.values)

In [ ]:
bert_embeddings = np.load("/content/drive/MyDrive/fyp/bert_embeddings.npy")

synthetic behavioral dataset

In [ ]:
import numpy as np
import pandas as pd

# Set random seed
np.random.seed(42)

# Generate synthetic behavioral data
behavioral_data = {
    "patient_id": speech_df["patient_id"],
    "reaction_time": np.random.uniform(1, 5, len(speech_df)),
    "task_accuracy": np.random.uniform(50, 100, len(speech_df)),
    "completion_time": np.random.uniform(5, 20, len(speech_df)),
    "attention_span": np.random.uniform(3, 10, len(speech_df)),  # New Feature: Attention Span (in minutes)
    "error_rate": np.random.uniform(0, 0.5, len(speech_df)),  # New Feature: Error Rate (0-0.5)
    "processing_speed": np.random.uniform(0.5, 2.5, len(speech_df)),  # New Feature: Processing Speed (Arbitrary Units)
}

behavior_df = pd.DataFrame(behavioral_data)

In [ ]:
# Normalize behavioral data
behavior_df["reaction_time"] = (behavior_df["reaction_time"] - behavior_df["reaction_time"].min()) / (behavior_df["reaction_time"].max() - behavior_df["reaction_time"].min())
behavior_df["task_accuracy"] /= 100
behavior_df["completion_time"] /= behavior_df["completion_time"].max()
behavior_df["attention_span"] /= behavior_df["attention_span"].max()
behavior_df["error_rate"] /= behavior_df["error_rate"].max()
behavior_df["processing_speed"] /= behavior_df["processing_speed"].max()

# Save to CSV
file_path = "enhanced_synthetic_behavioral_data.csv"
behavior_df.to_csv(file_path, index=False)

print(f"Dataset successfully saved as {file_path}")

Dataset successfully saved as enhanced_synthetic_behavioral_data.csv


In [ ]:
# Merge speech, text, and behavioral data
merged_df = speech_df.merge(behavior_df, on="patient_id")

# Define Features and Labels
X = merged_df.drop(columns=["patient_id", "label"])
y = merged_df["label"]

In [ ]:
print(speech_df["patient_id"].head())  # Sample IDs from speech dataset
print(behavior_df["patient_id"].head())  # Sample IDs from behavior dataset

0           Dan Ingram
1          Terry Jones
2    Maureen Forrester
3     Aileen Hernandez
4       Steve Lawrence
Name: patient_id, dtype: object
0           Dan Ingram
1          Terry Jones
2    Maureen Forrester
3     Aileen Hernandez
4       Steve Lawrence
Name: patient_id, dtype: object


In [ ]:
print(merged_df.head())

        mfcc       pitch  pause_count  speech_rate         patient_id  label  \
0  -7.922390  168.733675          1.0   323.000000         Dan Ingram      1   
1 -20.059917  160.364489          1.0   339.000000        Terry Jones      1   
2 -14.480901  182.225977          1.0   403.000000  Maureen Forrester      1   
3 -19.103064  193.957124          2.0   394.666667   Aileen Hernandez      1   
4 -10.819764  125.532001          1.0   631.000000     Steve Lawrence      1   

   text_dim_0  text_dim_1  text_dim_2  text_dim_3  ...  text_dim_764  \
0   -0.155559   -0.058839    0.312934    0.258014  ...     -0.026758   
1   -0.112149   -0.007126    0.097162    0.016589  ...     -0.012078   
2   -0.051789    0.169614    0.333880   -0.046509  ...     -0.155913   
3   -0.092661    0.144861    0.115789    0.036846  ...      0.047513   
4   -0.066994    0.031790    0.426530    0.136842  ...     -0.155909   

   text_dim_765  text_dim_766  text_dim_767  reaction_time  task_accuracy  \
0     -0.

In [ ]:
merged_df.to_csv("/content/drive/MyDrive/fyp/merged_data.csv", index=False)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# 1️⃣ Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2️⃣ Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 3️⃣ Random Forest Classifier (Hyperparameter Tuning)
param_grid_rf = {
    "n_estimators": [100, 200, 300],
    "max_depth": [10, 20, 30],
    "min_samples_split": [2, 5, 10],
}
rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=5, n_jobs=-1)
rf.fit(X_train, y_train)

# 4️⃣ XGBoost Classifier (Hyperparameter Tuning)
param_grid_xgb = {
    "learning_rate": [0.01, 0.1, 0.2],
    "n_estimators": [100, 200, 300],
    "max_depth": [3, 6, 9],
}
xgb = GridSearchCV(XGBClassifier(use_label_encoder=False, eval_metric="logloss"), param_grid_xgb, cv=5, n_jobs=-1)
xgb.fit(X_train, y_train)

# 5️⃣ Evaluate Models
y_pred_rf = rf.best_estimator_.predict(X_test)
y_pred_xgb = xgb.best_estimator_.predict(X_test)

acc_rf = accuracy_score(y_test, y_pred_rf)
acc_xgb = accuracy_score(y_test, y_pred_xgb)

print("🔥 Random Forest Accuracy:", acc_rf)
print("🔥 XGBoost Accuracy:", acc_xgb)
print("✅ Best RF Params:", rf.best_params_)
print("✅ Best XGB Params:", xgb.best_params_)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:22:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


🔥 Random Forest Accuracy: 0.7209302325581395
🔥 XGBoost Accuracy: 0.7906976744186046
✅ Best RF Params: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 200}
✅ Best XGB Params: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100}


In [ ]:
from sklearn.feature_selection import SelectFromModel
import xgboost as xgb

# Use the best XGBoost parameters you found
best_xgb_params = {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100}

# Train XGBoost for feature selection
xgb_clf = xgb.XGBClassifier(**best_xgb_params)
xgb_clf.fit(X_train, y_train)

# Select important features
selector = SelectFromModel(xgb_clf, threshold="median", prefit=True)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Train XGBoost again with selected features
xgb_final = xgb.XGBClassifier(**best_xgb_params)
xgb_final.fit(X_train_selected, y_train)

# Evaluate
y_pred_xgb_final = xgb_final.predict(X_test_selected)
acc_xgb_final = accuracy_score(y_test, y_pred_xgb_final)

print("🚀 Improved XGBoost Accuracy:", acc_xgb_final)

🚀 Improved XGBoost Accuracy: 0.7906976744186046


In [ ]:
!pip install stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 34.1 MB/s eta 0:00:00
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 1.1.0
    Uninstalling gymnasium-1.1.0:
      Successfully uninstalled gymnasium-1.1.0


In [ ]:
!pip install shimmy>=2.0

In [ ]:
import gym
import numpy as np
from gym import spaces
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env

class CognitiveTaskEnv(gym.Env):
    def __init__(self):
        super(CognitiveTaskEnv, self).__init__()

        # Cognitive state: reaction time, accuracy, engagement
        self.observation_space = spaces.Box(low=0, high=1, shape=(3,), dtype=np.float32)
        self.action_space = spaces.Discrete(3)  # Actions: Memory, Attention, Problem-Solving

        self.state = np.random.rand(3)  # Initialize with random cognitive states
        self.steps = 0
        self.max_steps = 50  # Increase training horizon

    def step(self, action):
        self.steps += 1
        reaction_time, accuracy, engagement = self.state

        # Action effects: More diverse and meaningful
        if action == 0:  # Memory Task
            accuracy += np.random.uniform(0.01, 0.08)
            engagement -= np.random.uniform(0.01, 0.03)
        elif action == 1:  # Attention Task
            engagement += np.random.uniform(0.02, 0.07)
            reaction_time -= np.random.uniform(0.01, 0.03)
        elif action == 2:  # Problem-Solving Task
            reaction_time -= np.random.uniform(0.02, 0.05)
            accuracy += np.random.uniform(0.01, 0.06)

        # Clip values to stay within valid range
        self.state = np.clip([reaction_time, accuracy, engagement], 0, 1)

        # Dynamic Reward Function
        reward = (accuracy * 0.6) + (engagement * 0.4) - (reaction_time * 0.3)

        # Episode termination condition
        done = (self.steps >= self.max_steps) or (accuracy >= 0.95 and engagement >= 0.95)

        return np.array(self.state, dtype=np.float32), reward, done, {}

    def reset(self):
        self.state = np.random.rand(3)
        self.steps = 0
        return np.array(self.state, dtype=np.float32)

# Train DQN Agent with adjusted hyperparameters
env = make_vec_env(CognitiveTaskEnv, n_envs=1)
model = DQN("MlpPolicy", env, verbose=1, learning_rate=0.0005, batch_size=64, buffer_size=100000)
model.learn(total_timesteps=20000)
model.save("dqn_cognitive_task")

# Test the trained model
env = CognitiveTaskEnv()
obs = env.reset()
for _ in range(50):
    action, _ = model.predict(obs)
    obs, reward, done, _ = env.step(action)
    print(f"Action: {action}, Reward: {reward:.3f}, New State: {obs}")
    if done:
        break

Using cuda device
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 49.2     |
|    ep_rew_mean      | 38.1     |
|    exploration_rate | 0.906    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1646     |
|    time_elapsed     | 0        |
|    total_timesteps  | 197      |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 0.207    |
|    n_updates        | 24       |
----------------------------------


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 45.9     |
|    ep_rew_mean      | 33       |
|    exploration_rate | 0.826    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1201     |
|    time_elapsed     | 0        |
|    total_timesteps  | 367      |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 0.0138   |
|    n_updates        | 66       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 47.2     |
|    ep_rew_mean      | 33.9     |
|    exploration_rate | 0.731    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 1053     |
|    time_elapsed     | 0        |
|    total_timesteps  | 567      |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 0.00357  |
|    n_updates      

In [ ]:
import joblib
from stable_baselines3 import DQN

# Save Dementia Classifier
joblib.dump(xgb, "/content/drive/MyDrive/fyp/dementia_model.pkl")

# Save DQN Cognitive Task Model
model.save("/content/drive/MyDrive/fyp/dqn_cognitive_task")

print("✅ Models saved successfully!")

✅ Models saved successfully!
